In [10]:
import spacy
import pandas as pd

In [1]:
from scispacy_fun.CustomSpacyF import  CustomSpacyF as csf

## load sicspacy's model

In [43]:
strVer = "10"
nlp = spacy.load("en_core_sci_lg")

##  get single word's vector from scispacy model

In [44]:
dicVector = sv.getVector(nlp,"psychosis")

for word,vec in dicVector.items():
    print(word,vec)

psychosis 3.1402366


##  get sentence's vector from scispacy model


In [45]:
print(sv.getSentenceVector(nlp,"heart type disease"))
print(sv.getSentenceVector(nlp,"neurological type disease"))

['heart type disease', 1.8547993]
['neurological type disease', 1.7834562]


## connect to mimic db

In [ ]:


host = "203.64.95.185"
dbname = "postgres"
user = "nkust1108"
password = "nkust1108"

In [6]:
def getConnString(path:str = './db_conn.txt')->dict:
    dic_conn_str = {}

    f = open('./db_conn.txt', 'r')

    lst_tmp = []
    for line in f.readlines():
        lst_tmp = line.split(':')
        dic_conn_str[lst_tmp[0]] = lst_tmp[1].replace('\n','')

    f.close()
    del lst_tmp

    print(dic_conn_str)
    return dic_conn_str

In [46]:
import psycopg2

dic = getConnString(path='./db_conn.txt')

conn_string = f"host={dic['host']} dbname={dic['dbname']} user={user['user']} password={password['password']}"

conn = psycopg2.connect(conn_string)
cursor = conn.cursor()

def excuteSqlToDict(cursor,strSql):
    dic = {}

    strKey = ""
    strVal = ""

    cursor.execute(strSql)

    for row in cursor:
        strKey = row[0].strip()
        strVal = row[1].strip()
        dic[strKey] = strVal
    return dic


## get disease feature(long title) by icd code from mimic db

In [47]:
def excuteSqlToDict(cursor,strSql):
    dic = {}

    strKey = ""
    strVal = ""

    cursor.execute(strSql)

    for row in cursor:
        strKey = row[0].strip()
        strVal = row[1].strip()
        dic[strKey] = strVal
    return dic


In [48]:
def getIcdCodeByVersion(cursor,icd_code_list):
    strSql = """
        select
            icd_code,long_title 
        from 
            mimic_hosp.d_icd_diagnoses
        where
            icd_version = """+ strVer +"""
        and
            icd_code in (""" + icd_code_list + """)
        order by icd_code asc
    """

    return excuteSqlToDict(cursor,strSql)



## get input data by funtion: "getIcdCodeByVersion"

In [49]:
lstIcdCode = []
f = open('/Users/alros/lab/nlp/word2vec/疾病轉word2vec/mimic轉word2vec/icdCodeBySubId_10.txt')

strIcdCodeList = ""
for s in f.readlines():
    lst = s.split(":")
    if len(lst)== 2:
        lst = lst[1].split(",")
        for icdCode in lst:
            
            strTmp = icdCode.strip() 
            if strTmp not in strIcdCodeList:
                strIcdCodeList = strIcdCodeList + "'" + strTmp + "',"
                lstIcdCode.append(strTmp)
            # else:
            #     print(icdCode)

strIcdCodeList = strIcdCodeList[0:len(strIcdCodeList)-1]

print(len(strIcdCodeList.split(",")))
print(len(lstIcdCode))

11362
11362


In [50]:
dicIcdInfo = getIcdCodeByVersion(cursor,strIcdCodeList)

In [51]:
print(len(dicIcdInfo))
for key,val in dicIcdInfo.items():
    print(key,val)
    break


11362
A0100 Typhoid fever, unspecified


## get disease feature's vector and export to csv

In [52]:
dicVector = {}
dicTmp = {}
aryVec = []

from datetime import datetime
strDate = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
strFileName = "icd"+strVer+"_vec_" + strDate + ".txt"

f = open(strFileName,"w")

intCount = 0
for icdCode,longTitle in dicIcdInfo.items():

    aryVec = sv.getSentenceVector(nlp,longTitle,blnStayComma = False,blnStayBrackets = False)

    strNewTitle = aryVec[0]
    odjVec = aryVec[1]

    #s = icdCode + "," + strNewTitle + "," + str(odjVec)
    s = icdCode + "," + str(odjVec)
    print(intCount,s)
    if intCount < 10:
        f.write(s + "\n")
    intCount += 1

print('count:',intCount)

f.close()

    

0 A0100,2.4891295
1 A020,2.8890347
2 A021,2.5600061
3 A031,2.065963
4 A033,2.1304424
5 A039,2.12078
6 A044,2.0150425
7 A045,3.241548
8 A046,2.1477146
9 A047,2.0599933
10 A0471,1.8407509
11 A0472,1.3494363
12 A048,1.4695779
13 A049,1.6771362
14 A059,1.9523838
15 A071,3.0637524
16 A0811,1.5242687
17 A082,2.7939095
18 A0839,1.8531806
19 A084,1.7291449
20 A088,1.4876279
21 A09,1.8685048
22 A150,1.632827
23 A156,3.4744468
24 A157,2.0395935
25 A159,2.054448
26 A1801,1.7536914
27 A182,2.5609367
28 A184,1.2995173
29 A1883,1.3218768
30 A1884,1.6652166
31 A199,2.4703078
32 A227,2.4545412
33 A280,2.2408414
34 A281,2.8579998
35 A310,2.162772
36 A311,2.1251757
37 A312,3.0981383
38 A318,1.883366
39 A319,1.8037512
40 A369,2.3536835
41 A3790,1.798504
42 A390,3.209299
43 A392,2.8951068
44 A393,2.8670967
45 A400,1.4106176
46 A401,1.4219588
47 A403,1.9784886
48 A408,2.0125196
49 A409,2.0808961
50 A4101,2.2878249
51 A4102,2.3144996
52 A411,1.2418661
53 A412,1.4900049
54 A413,2.2429922
55 A414,1.5532471
56